In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from pydataset import data

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import sklearn as sk
import env

# filter out warnings
import warnings
warnings.filterwarnings('ignore')

# our own scripts
import acquire
import prepare

### Model Evaluation Exercises



2. Given the following confusion matrix, evaluate (by hand) the model's performance.


|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |

In [108]:
# positive case = dog
# negative case = cat

In [2]:
# Accuracy:
accuracy = (46 + 34)/(46 + 7 + 13 + 34)
accuracy

0.8

In [3]:
# Precision:
precision = 46/(46+7)
precision

0.8679245283018868

In [4]:
# Recall:
recall = 46/(46+13) 
recall

0.7796610169491526

#### In the context of this problem, what is a false positive?

Its actually a dog but we predicted it as a cat.

#### In the context of this problem, what is a false negative?

Its actually a cat but we predicted a dog.

#### How would you describe this model?

In [110]:
# If every instance was a dog:
actual_dog = 53 
actual_cat = 47
pred_dog = 59
pred_cat = 41
baseline = 100

# model accuracy:(TP + TN)/(TP+TN+FP+FN)
model_accuracy = (46 + 34)/(46 + 7 + 13 + 34)

# baseline accuracy:
baseline_accuracy = 46/100

# model_precision:(TP)/(TP+FP)
model_precision = 46/(46+13)
      
#baseline_precison:
baseline_precision = 46/100
      
# model_recall:(TP)/(TP+)
model_recall = 46/(46+7)
      
#baseline_precison: (zero FN because all positive)
baseline_recall = 46/46
      
print(f'   model accuracy: {model_accuracy}')
print(f'baseline accuracy: {baseline_accuracy}')
print()
print(f'   model recall: {model_recall}')
print(f'baseline recall: {baseline_recall}')
print()
print(f'model precision: {model_precision}')
print(f'baseline precision: {baseline_precision}')

   model accuracy: 0.8
baseline accuracy: 0.46

   model recall: 0.8679245283018868
baseline recall: 1.0

model precision: 0.7796610169491526
baseline precision: 0.46


 The model is overall accurate at 80% (34% more than the baseline). It airs on the side of guessing dog(a slightly higher recall rate) at the cost of less accurate dog predictions(lower precision of 78%).

### 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

#### Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

#### Use the predictions dataset and pandas to help answer the following questions:

In [118]:
df = pd.read_csv('c3.csv')
df.head()

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect


In [119]:
df.describe()

,actual,model1,model2,model3
count,200,200,200,200
unique,2,2,2,2
top,No Defect,No Defect,No Defect,No Defect
freq,184,190,110,101


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   actual  200 non-null    object
 1   model1  200 non-null    object
 2   model2  200 non-null    object
 3   model3  200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB


#### An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [121]:
df.actual.value_counts()

No Defect    184
Defect        16
Name: actual, dtype: int64

TP = Defect identified, defect present
TN = No defect identified, no defect present
FP = Defect indentified, no defect present
FN = No defect identified, defect present

- **Which evaluation metric should be used?**
 Recall with FN having the most impact(expense)

- **Which model would be the best fit for this use case?**
 Model3 would be best fit with it having the most defects identified.

#### Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [127]:
df['baseline'] = 'Defect'

model1 = df[['actual', 'model1']]
model1['baseline'] = model1.actual.mode()[0]

model2 = df[['actual', 'model2']]
model2['baseline'] = model2.actual.mode()[0]

model3 = df[['actual', 'model3']]
model3['baseline'] = model3.actual.mode()[0]

In [128]:
# Model 1
positive = 'Defect'

# accuracy -- overall hit rate
model_accuracy = (df.model1 == df.actual).mean()
baseline_accuracy = (df.baseline == df.actual).mean()
# precision -- how good are our positive predictions?
# precision -- model performance | predicted positive
subset = df[df.model1 == positive]
model_precision = (subset.model1 == subset.actual).mean()
subset = df[df.baseline == positive]
baseline_precision = (subset.baseline == subset.actual).mean()
# recall -- how good are we at detecting actual positives?
# recall -- model performance | actual positive
subset = df[df.actual == positive]
model_recall = (subset.model1 == subset.actual).mean()
baseline_recall = (subset.baseline == subset.actual).mean()


In [129]:
# Model 2
positive = 'Defect'

# accuracy -- overall hit rate
model_accuracy = (df.model2 == df.actual).mean()
baseline_accuracy = (df.baseline == df.actual).mean()
# precision -- how good are our positive predictions?
# precision -- model performance | predicted positive
subset = df[df.model2 == positive]
model_precision = (subset.model2 == subset.actual).mean()
subset = df[df.baseline == positive]
baseline_precision = (subset.baseline == subset.actual).mean()
# recall -- how good are we at detecting actual positives?
# recall -- model performance | actual positive
subset = df[df.actual == positive]
model_recall = (subset.model2 == subset.actual).mean()
baseline_recall = (subset.baseline == subset.actual).mean()


In [130]:
# Model 3
positive = 'Defect'

# accuracy -- overall hit rate
model_accuracy = (df.model3 == df.actual).mean()
baseline_accuracy = (df.baseline == df.actual).mean()
# precision -- how good are our positive predictions?
# precision -- model performance | predicted positive
subset = df[df.model3 == positive]
model_precision = (subset.model3 == subset.actual).mean()
subset = df[df.baseline == positive]
baseline_precision = (subset.baseline == subset.actual).mean()
# recall -- how good are we at detecting actual positives?
# recall -- model performance | actual positive
subset = df[df.actual == positive]
model_recall = (subset.model3 == subset.actual).mean()
baseline_recall = (subset.baseline == subset.actual).mean()


In [131]:
print('==========================')
print('Model 1:')
print('--------------------------')
print(f'   model accuracy: {model_accuracy1:.2%}')
print(f'baseline accuracy: {baseline_accuracy1:.2%}')
print('--------------------------')
print(f'   model recall: {model_recall1:.2%}')
print(f'baseline recall: {baseline_recall1:.2%}')
print('--------------------------')
print(f'model precision: {model_precision1:.2%}')
print(f'baseline precision: {baseline_precision1:.2%}')
print('==========================')
print('Model 2:')
print('--------------------------')
print(f'   model accuracy: {model_accuracy2:.2%}')
print(f'baseline accuracy: {baseline_accuracy2:.2%}')
print('--------------------------')
print(f'   model recall: {model_recall2:.2%}')
print(f'baseline recall: {baseline_recall2:.2%}')
print('--------------------------')
print(f'model precision: {model_precision2:.2%}')
print(f'baseline precision: {baseline_precision2:.2%}')
print('==========================')
print('Model 3:')
print('--------------------------')
print(f'   model accuracy: {model_accuracy3:.2%}')
print(f'baseline accuracy: {baseline_accuracy3:.2%}')
print('--------------------------')
print(f'   model recall: {model_recall3:.2%}')
print(f'baseline recall: {baseline_recall3:.2%}')
print('--------------------------')
print(f'model precision: {model_precision3:.2%}')
print(f'baseline precision: {baseline_precision3:.2%}')
print('==========================')

Model 1:
--------------------------
   model accuracy: 80.74%
baseline accuracy: 65.08%
--------------------------
   model recall: 80.33%
baseline recall: 100.00%
--------------------------
model precision: 80.33%
baseline precision: 100.00%
Model 2:
--------------------------
   model accuracy: 63.04%
baseline accuracy: 65.08%
--------------------------
   model recall: 49.08%
baseline recall: 100.00%
--------------------------
model precision: 49.08%
baseline precision: 100.00%
Model 3:
--------------------------
   model accuracy: 50.96%
baseline accuracy: 65.08%
--------------------------
   model recall: 50.86%
baseline recall: 100.00%
--------------------------
model precision: 50.86%
baseline precision: 100.00%


In [78]:
# models = [model1, model2, model3]
# positive = 'Defect'

# for i, model in enumerate(models):
    
#     model_number = i + 1
    
#     print(f'Model #{model_number}:\n')
#     evaluate_classifier(model, positive)
#     print('--------------------------')

Situation calls for precision because the cost of having an undetected defect is much higher than identifying a defect where there isn't one.

Use model 1 because we want to identify as many positives, both true and false, as possible.

### 4.) You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).


In [133]:
df = pd.read_csv('gives_you_paws.csv')
df.head()

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog


#### a. In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?

In [134]:
df['baseline'] = 'dog'

model1 = df[['actual', 'model1', 'baseline']]
model1['baseline'] = model1.actual.mode()[0]

model2 = df[['actual', 'model2', 'baseline']]
model2['baseline'] = model2.actual.mode()[0]

model3 = df[['actual', 'model3', 'baseline']]
model3['baseline'] = model3.actual.mode()[0]

model4 = df[['actual', 'model4', 'baseline']]
model4['baseline'] = model4.actual.mode()[0]


In [135]:
# Model 1
positive = 'dog'

# accuracy -- overall hit rate
model_accuracy1 = (df.model1 == df.actual).mean()
baseline_accuracy1 = (df.baseline == df.actual).mean()
# precision -- how good are our positive predictions?
# precision -- model performance | predicted positive
subset1 = df[df.model1 == positive]
model_precision1 = (subset.model1 == subset.actual).mean()
subset1 = df[df.baseline == positive]
baseline_precision1 = (subset.baseline == subset.actual).mean()
# recall -- how good are we at detecting actual positives?
# recall -- model performance | actual positive
subset1 = df[df.actual == positive]
model_recall1 = (subset.model1 == subset.actual).mean()
baseline_recall1 = (subset.baseline == subset.actual).mean()


In [136]:
# Model 2
positive = 'dog'

# accuracy -- overall hit rate
model_accuracy2 = (df.model2 == df.actual).mean()
baseline_accuracy2 = (df.baseline == df.actual).mean()
# precision -- how good are our positive predictions?
# precision -- model performance | predicted positive
subset2 = df[df.model2 == positive]
model_precision2 = (subset.model2 == subset.actual).mean()
subset2 = df[df.baseline == positive]
baseline_precision2 = (subset.baseline == subset.actual).mean()
# recall -- how good are we at detecting actual positives?
# recall -- model performance | actual positive
subset2 = df[df.actual == positive]
model_recall2 = (subset.model2 == subset.actual).mean()
baseline_recall2 = (subset.baseline == subset.actual).mean()


In [137]:
# Model 3
positive = 'dog'

# accuracy -- overall hit rate
model_accuracy3 = (df.model3 == df.actual).mean()
baseline_accuracy3 = (df.baseline == df.actual).mean()
# precision -- how good are our positive predictions?
# precision -- model performance | predicted positive
subset3 = df[df.model3 == positive]
model_precision3 = (subset.model3 == subset.actual).mean()
subset3 = df[df.baseline == positive]
baseline_precision3 = (subset.baseline == subset.actual).mean()
# recall -- how good are we at detecting actual positives?
# recall -- model performance | actual positive
subset3 = df[df.actual == positive]
model_recall3 = (subset.model3 == subset.actual).mean()
baseline_recall3 = (subset.baseline == subset.actual).mean()


In [143]:
# Model 4
positive = 'dog'

# accuracy -- overall hit rate
model_accuracy4 = (df.model4 == df.actual).mean()
baseline_accuracy4 = (df.baseline == df.actual).mean()
# precision -- how good are our positive predictions?
# precision -- model performance | predicted positive
subset4 = df[df.model4 == positive]
model_precision4 = (subset.model4 == subset.actual).mean()
subset4 = df[df.baseline == positive]
baseline_precision4 = (subset.baseline == subset.actual).mean()
# recall -- how good are we at detecting actual positives?
# recall -- model performance | actual positive
subset4 = df[df.actual == positive]
model_recall4 = (subset.model4 == subset.actual).mean()
baseline_recall4 = (subset.baseline == subset.actual).mean()


AttributeError: 'DataFrame' object has no attribute 'model4'

In [141]:
print('============================')
print('MODEL 1:')

print(f'   model accuracy: {model_accuracy1:.2%}')
print(f'baseline accuracy: {baseline_accuracy1:.2%}')
print()
print(f'   model recall: {model_recall1:.2%}')
print(f'baseline recall: {baseline_recall1:.2%}')
print()
print(f'model precision: {model_precision1:.2%}')
print(f'baseline precision: {baseline_precision1:.2%}')
print('============================')
print('MODEL 2:')

print(f'   model accuracy: {model_accuracy2:.2%}')
print(f'baseline accuracy: {baseline_accuracy2:.2%}')
print()
print(f'   model recall: {model_recall2:.2%}')
print(f'baseline recall: {baseline_recall2:.2%}')
print()
print(f'model precision: {model_precision2:.2%}')
print(f'baseline precision: {baseline_precision2:.2%}')
print('============================')
print('MODEL 3:')

print(f'   model accuracy: {model_accuracy3:.2%}')
print(f'baseline accuracy: {baseline_accuracy3:.2%}')
print()
print(f'   model recall: {model_recall3:.2%}')
print(f'baseline recall: {baseline_recall3:.2%}')
print()
print(f'model precision: {model_precision3:.2%}')
print(f'baseline precision: {baseline_precision3:.2%}')
print('============================')
print('MODEL 4:')

print(f'   model accuracy: {model_accuracy4:.2%}')
print(f'baseline accuracy: {baseline_accuracy4:.2%}')
print()
print(f'   model recall: {model_recall4:.2%}')
print(f'baseline recall: {baseline_recall4:.2%}')
print()
print(f'model precision: {model_precision4:.2%}')
print(f'baseline precision: {baseline_precision4:.2%}')
print('============================')

MODEL 1:
   model accuracy: 80.74%
baseline accuracy: 65.08%

   model recall: 50.00%
baseline recall: 100.00%

model precision: 50.00%
baseline precision: 100.00%
MODEL 2:
   model accuracy: 63.04%
baseline accuracy: 65.08%

   model recall: 56.25%
baseline recall: 100.00%

model precision: 56.25%
baseline precision: 100.00%
MODEL 3:
   model accuracy: 50.96%
baseline accuracy: 65.08%

   model recall: 81.25%
baseline recall: 100.00%

model precision: 81.25%
baseline precision: 100.00%
MODEL 4:
   model accuracy: 74.26%
baseline accuracy: 65.08%

   model recall: 95.57%
baseline recall: 100.00%

model precision: 95.57%
baseline precision: 100.00%


#### Model 1 is the best for accuracy.

#### b. Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recomend for Phase I? For Phase II?

positive = dog is correctly identified
negative = its a cat

TP: Identified as dog, its a dog
TN: Identified as a cat, its a cat
FP: Identified as dog, its a cat
FN: Identified as a cat, its a dog

#### At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

#### Phase I:
 Using Recall. Want as many positives as possible. Use Model 4.

#### Phase II: 
 Using Precision. Want to sure all decisons are as correct as possible. Use Model 4 as well.

#### c. Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recomend for Phase I? For Phase II?

positive = cat is correctly identified
negative = its a dog

TP: Identified as cat, its a cat
TN: Identified as a dog, its a dog
FP: Identified as cat, its a dog
FN: Identified as a dog, its a cat

#### Phase I:
 

#### Phase II:
 